In [ ]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#data handling
import pandas as pd
import numpy as np
import calendar
import datetime

#plotting
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

#time series analysis and modelling
from scipy.ndimage import gaussian_filter
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.tsa.stattools import adfuller,kpss

# EDA

## Data Loading

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_store = pd.read_csv('data/store.csv')

In [ ]:
df_train.head()

In [ ]:
# function to rename columns in lower case
def lower_case(dataframe):
    cols = dataframe.columns.tolist()
    cols = [col.lower() for col in cols]
    dataframe.columns = cols
    return dataframe

In [ ]:
lower_case(df_train);
lower_case(df_store);
lower_case(df_test);

In [ ]:
# function to change date into datetime
def to_datetime(dataframe):
    dataframe.assign(
        timestamp = lambda x: pd.to_datetime(x['date']),
        year = lambda x: x['timestamp'].dt.year,
        month = lambda x: x['timestamp'].dt.month,
        day = lambda x: x['timestamp'].dt.day,
        dayofyear = lambda x: x['timestamp'].dt.dayofyear)
    return dataframe
# does not work :( 

In [ ]:
#df_train = to_datetime(df_train)

In [ ]:
# changing date into datetime object, inserting year, month, day and dayofyear columns
df_train = df_train.assign(
            timestamp = lambda x: pd.to_datetime(x['date']),
            year = lambda x: x['timestamp'].dt.year,
            month = lambda x: x['timestamp'].dt.month,
            day = lambda x: x['timestamp'].dt.day,
            dayofyear = lambda x: x['timestamp'].dt.dayofyear)
df_train.drop("date", inplace=True, axis=1)

In [ ]:
df_train.head()

In [ ]:
print(df_store.shape)
df_store.head()

## Data Cleaning

In [ ]:
df_train.shape

### Closed stores and zero sales stores

In [ ]:
# closed stores
df_train[(df_train['open'] == 0) & (df_train['sales'] == 0)].shape

In [ ]:
df_train[(df_train['open'] != 0) & (df_train['sales'] == 0)].shape

There are 172817 stores, which were closed and had no sales. In addition to 54 open stores, which had no sales at that day.

We cannot make any predictions for stores, which were closed, and stores which were open, but had no sales, might have had external influences, such as remodeling.

To avoid any bias, we should drop these datapoints with 0 sales.

In [ ]:
df_train = df_train[(df_train["open"] != 0) & (df_train['sales'] != 0)]
df_train.shape

In [ ]:
df_train['stateholiday'].unique()

In [ ]:
df_train.stateholiday.value_counts()

In [ ]:
df_train['stateholiday'].replace({0:'0'}, inplace=True)
round(df_train.describe().T,2)

### Cleaning NaN's

In [ ]:
df_store.isnull().sum()

In [ ]:
df_store[pd.isnull(df_store.competitiondistance)]

In [ ]:
# fill NaN with a median value
df_store['competitiondistance'].fillna(df_store['competitiondistance'].median(), inplace = True)
df_store['competitiondistance'].isnull().sum()

In [ ]:
tmp = df_store[pd.isnull(df_store.competitionopensinceyear)]
tmp[tmp.competitiondistance != 0].shape

Here these stores have a competition in their vicinity ('competitiondistance' =/= 0), but there is no information about the year this competition has been open. This value needs to be imputed in a meaningful way. Or just filled with '0'.

In [ ]:
tmp = df_store[pd.isnull(df_store.promo2sinceweek)]
tmp[tmp.promo2 != 0].shape

There are no stores with information about 'promo2sinceweek' which have 'NaN' in promo2.

In [ ]:
# replace NA's by 0
df_store.fillna(0, inplace = True)

In [ ]:
print(df_store.isnull().sum())
print('------------------------')
print(df_train.isnull().sum())

## Data Exploration

In [ ]:
# Merge df_store and df_train
df = df_train.merge(df_store, how='left', left_on=df_train.store, right_on=df_store.store)
df.drop(['key_0', 'store_y'], axis=1, inplace=True)
df = df.rename(columns={'store_x':'store'})

In [ ]:
df.groupby('storetype')['sales'].describe()

In [ ]:
df.groupby('storetype')['customers', 'sales'].sum()

In [ ]:
# sales trends
#sns.factorplot(data = df, x = 'month', y = "sales", 
#               col = 'storetype',
#               palette = 'plasma',
#               hue = 'storetype',
#               row = 'promo', 
#               ) 

Storetype B has the highest sales numbers, with the largest variance. All storetypes show increased sales numbers towards christmas. 

Stores which have run a promo, show higher sales. But storetypes a,c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo.

In [ ]:
# customers trends
#sns.factorplot(data = df, x = 'month', y = "customers", 
#               col = 'storetype',
#               palette = 'plasma',
#               hue = 'storetype',
#               row = 'promo',
#               ) 

Storetype B has the highest number of customers, with the largest variance. All storetypes show an increase of customers towards christmas. This trend is higher, if they have run a promo.

Same effect of a dip for storetypes a,c and d in customers towards easter can be also be seen here.

In [ ]:
# sale per customer trends
df['salepercustomer'] = df['sales']/df['customers']
#sns.factorplot(data = df, x = 'month', y = "salepercustomer", 
#               col = 'storetype',
#               palette = 'plasma',
#               hue = 'storetype',
#               row = 'promo', 
#               ) 

Sales per customer:
storetype b seems to be where customers only buy small items in low numbers (possible trainstation location?)
storetype d customers buy the largest quantity
a und c are very similar

In [ ]:
# weekday trends
#sns.factorplot(data = df, x = 'dayofweek', y = "sales", 
#               col = 'storetype',
#               palette = 'plasma',
#               hue = 'storetype',
#               row = 'promo',
#               ) 

Similar trends regarding sales numbers and customers.
Highest number of sales and customers on mondays, if a promo was run

In [ ]:
# weekday customer trends
#sns.factorplot(data = df, x = 'dayofweek', y = "customers", 
#               col = 'storetype',
#               palette = 'plasma',
#               hue = 'storetype',
#               row = 'promo',
#               ) 

Promos are run only during the work-week, no promo on saturday/sunday.

Storetype b also open on sundays -> trainstation, fo sho
storetyp a lower number of customers on saturday, c and d increased

### Conclusion:
- Promos are run only during the work-week, no promo on saturday/sunday\n",
- Storetype B has the highest number of customers, with the largest variance\n",
- Storetype B has the highest sales numbers, with the largest variance\n",
- All storetypes show increased sales numbers towards christmas\n",
- Stores which have run a promo, show higher sales. But storetypes a, c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo."

In [ ]:
df.describe().round(2).T

In [ ]:
# Plot CompetitionDistance Vs Sales
df.plot(kind='scatter',x='competitiondistance',y='sales', figsize=(15,4))
df.plot(kind='box', y='competitiondistance', figsize=(15,4))

75% of all stores have their nearest competitor at under 7km distance, while 50% of all stores have their nearest competitor at 2.3km.

In [ ]:
sns.boxplot(data= df, x= 'storetype', y= 'competitiondistance', palette = 'plasma_r', order=["a", "b", "c", "d"])

Storetype B has the nearest competitors, while storetyp D has the largest range and the largest mean distance. The farthest away competitors belong to Storetype A.

In [ ]:
temp_df = df.groupby(df.storetype).sum()
sns.barplot(temp_df.index, temp_df.sales, palette='Blues');

In [ ]:
# plotting sales per storetype, based on promo1 or promo2
#g = sns.FacetGrid(data = df,
#                  col = 'promo',
#                  row = 'promo2',
#                  palette = 'plasma',
#                  #hue = 'assortment',
#                 )
#g.map_dataframe(sns.barplot, "storetype", "sales", order=["a","b","c","d"])

In [ ]:
# plotting counts per storetype, based on promo1 or promo2
#g = sns.FacetGrid(data = df, 
#               col = 'promo',
#                  row = 'promo2',
#               palette = 'plasma',
               #hue = 'assortment',
#               )
#g.map_dataframe(sns.countplot, "storetype", order=["a","b","c","d"])
#g.add_legend()

### Datatype and Encoding of Features

# Time Series Analysis

Plotting the seasonal decomposition of the sales per date. For this, the data for each day will be summed up and grouped by.

Afterwards, the model will decompose the sale values in an additive manner.

In [ ]:
tmp_df = df.groupby(df['timestamp']).sum()
season_decomp = seasonal_decompose(tmp_df['sales'], model='additive', freq=52)

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(18,8))
ax1.plot(season_decomp.trend)
ax1.axhline(y = tmp_df['sales'].mean(), color = 'r', linestyle = '-', label='Sales Mean')
ax1.set_title("Trend")
ax2.plot(season_decomp.resid)
ax2.set_title("Residuals")
ax1.legend()
plt.show()

The beginning of 2014 and 2015 show higher than average sales numbers. Especially the peak at the beginning of 2014 is very high.

During the second half of 2014, the sales drop significantly under the average value.

In [ ]:
tmp_df = df.copy()
tmp_df = tmp_df[tmp_df['year']==2014]
tmp_df = tmp_df.groupby(tmp_df['month']).sum()

plt.title('Promos done in 2014')
sns.lineplot(data=tmp_df, x=tmp_df.index, y=tmp_df['promo'], palette='Blues', label='Promo1')
sns.lineplot(data=tmp_df, x=tmp_df.index, y=tmp_df['promo2'], palette='Blues', label='Promo2')
plt.legend()
plt.show()

The downwards trend of sales at the second half of 2014 until Christmas time seems to coincide with the decreasing number of promos during that period.

In [ ]:
tmp_df = df.groupby(df['timestamp']).sum()

In [ ]:
#freq = 7(weekly), 30(monthly), 365(yearly)
weekly_decomp = seasonal_decompose(tmp_df['sales'], model='additive', freq=7)
monthly_decomp = seasonal_decompose(tmp_df['sales'], model='additive', freq=30)
yearly_decomp = seasonal_decompose(tmp_df['sales'], model='additive', freq=365)

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(18,8))
ax1.plot(weekly_decomp.trend)
ax1.axhline(y = tmp_df['sales'].mean(), color = 'r', linestyle = '-', label='Sales Mean')
ax1.set_title("Weekly Trend")
ax2.plot(monthly_decomp.trend)
ax2.axhline(y = tmp_df['sales'].mean(), color = 'r', linestyle = '-', label='Sales Mean')
ax2.set_title("Monthly Trend")
ax3.plot(yearly_decomp.trend)
ax3.axhline(y = tmp_df['sales'].mean(), color = 'r', linestyle = '-', label='Sales Mean')
ax3.set_title("Yearly Trend")
fig.tight_layout() 
plt.show()

In [ ]:
# Plot autocorrelation
f, (ax1, ax2) = plt.subplots(2, figsize = (12, 6))
plot_acf(tmp_df['sales'], lags = 50, ax=ax1)
plot_pacf(tmp_df['sales'], lags = 50, ax=ax2)

Those plots are showing the correlation of the series with itself, lagged by x time units correlation of the series with itself, lagged by x time units.

There is at two things common: non randomnes of the time series and high lag-1 (which will probably need a higher order of differencing d/D).

There is a weekly trend with positives spikes at the 7(s), 14(2s), 21(3s) and 28(4s) lags.


# Feature Engineering

In this section, varios categorical features will be either one-hot encoded(nominal feature) or label encoded (ordinal data). In addition, features regarding holiday and promo intervals will also be added.

In [ ]:
print(df.shape)
df.head()

First, the test data-set needs to be cleaned and features converted

In [ ]:
# changing date into datetime object, inserting year, month, day and dayofyear columns
df_test = df_test.assign(
            timestamp = lambda x: pd.to_datetime(x['date']),
            year = lambda x: x['timestamp'].dt.year,
            month = lambda x: x['timestamp'].dt.month,
            day = lambda x: x['timestamp'].dt.day,
            dayofyear = lambda x: x['timestamp'].dt.dayofyear)
df_test.drop("date", inplace=True, axis=1)

In [ ]:
# check for correct assignment of 'stateholiday' values
print(df_test['stateholiday'].unique())
print(df_test['stateholiday'].value_counts())

In [ ]:
# check for closed stores
df_test[df_test["open"].isnull()]

All of these store have no information for 'open', although these days are not a holiday ('stateholiday =/= 1) and are not affected by the closure of schools. They should be open.

In [ ]:
df_test['open'].fillna(1, inplace=True)
df_test['open']= df_test['open'].astype(int)

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train['is_train'] = 1
df_test['is_train'] = 0

In [ ]:
print(df_train.columns)
print('-----------------')
print(df_test.columns)
print('-----------------')
print(df_store.columns)

In [ ]:
df_model = pd.concat([df_train, df_test])

In [ ]:
df_model.head()

### Feature engineering from store features

In [ ]:
features_x = ['store', 'timestamp', 'dayofweek', 'open', 'promo', 'schoolholiday', 'stateholiday']
features_y = ['saleslog']

In [ ]:
# encoding categorical features
df_model['stateholiday'] = LabelEncoder().fit_transform(df_model['stateholiday']) 
df_store['storetype'] = LabelEncoder().fit_transform(df_store['storetype'])
df_store['assortment'] = LabelEncoder().fit_transform(df_store['assortment'])

Promo interval feature engineering

In [ ]:
# splitting 'Promointerval' string into individual strings and get the month
prom_interval = df_store['promointerval'].str.split(',').apply(pd.Series)

In [ ]:
prom_interval.columns = prom_interval.columns.map(lambda x: str(x) + '_prominterval')
df_store = df_store.join(prom_interval)

In [ ]:
def monthToNum(value):
    if(value=='Sept'):
        value='Sep'
    return list(calendar.month_abbr).index(value)
#mapping month abbr to month number
df_store['0_prominterval'] = df_store['0_prominterval'].map(lambda x: monthToNum(x) if str(x) != 'nan' else np.nan)
df_store['1_prominterval'] = df_store['1_prominterval'].map(lambda x: monthToNum(x) if str(x) != 'nan' else np.nan)
df_store['2_prominterval'] = df_store['2_prominterval'].map(lambda x: monthToNum(x) if str(x) != 'nan' else np.nan)
df_store['3_prominterval'] = df_store['3_prominterval'].map(lambda x: monthToNum(x) if str(x) != 'nan' else np.nan)

Promo feature engineering

In [ ]:
promo = []
for index, value in df_store[['promo2sinceweek', 'promo2sinceyear']].iterrows():
    try:
        year, week = int(value['promo2sinceyear']), int(value['promo2sinceweek'])
        date = pd.to_datetime("{}-{}-01".format(year, week), format='%Y%W')
        promo.append(date)
    except:
        promo.append(np.nan)
promo = pd.to_datetime(pd.Series(promo))
promo.shape

In [ ]:
df_store['promosince'] = promo #converted int to datetime
df_store['promosince'] = df_store.promosince.dt.strftime('%Y%m%d')

Competition feature engineering

In [ ]:
competition_open = []
for index, value in df_store[['competitionopensincemonth', 'competitionopensinceyear']].iterrows():
    try:
        year, month = int(value['competitionopensinceyear']), int(value['competitionopensincemonth'])
        date = pd.to_datetime("{}-{}-01".format(year, month), format='%Y-%m')
        competition_open.append(date)
    except:
        competition_open.append(np.nan)
competition_open = pd.Series(competition_open)
competition_open.shape

In [ ]:
df_store['competitionopen'] = competition_open #converted int to datetime
df_store['competitionopen'] = df_store['competitionopen'].dt.strftime('%Y%m%d')

This concludes the feature engineering from df_store.
The newly created features are put into store_features

In [ ]:
store_features = ['store', 'storetype', 'assortment', 'competitiondistance', 'competitionopen', 
                  'promosince', '0_prominterval']

In [ ]:
df_model = pd.merge(df_model, df_store[store_features], how='left', on=['store'])

In [ ]:
features_x

In [ ]:
# put new features into feature-list
features_x = list(set(features_x + store_features))

for feature in features_x:
    df_model[feature] = df_model[feature].fillna(-999) #out of range value for model

In [ ]:
df_model['dateint'] = df_model.timestamp.dt.strftime('%Y%m%d').map(int) #mapping to Int
df_model['competitionopen'] = df_model.competitionopen.map(int)
df_model['promosince'] = df_model.promosince.map(int)

### Feature engineering from df_train features

Holiday feature engineering

In [ ]:
holidays_next_week=[]
holidays_next_week_index=[]
for index, value in df_model.groupby(df_model['timestamp']).sum().iterrows():
    start_range = index + datetime.timedelta(days=7)
    end_range = index + datetime.timedelta(days=15)
    school_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).schoolholiday)
    state_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).stateholiday)
    holidays_next_week.append(school_holidays+state_holidays)
    holidays_next_week_index.append(index)
    
holidays_next_week = pd.Series(holidays_next_week)
holidays_next_week.shape

In [ ]:
holidays_this_week=[]
index_list = []
for index, value in df_model.groupby(df_model['timestamp']).sum().iterrows():
    start_range = index 
    end_range = index + datetime.timedelta(days=7)
    school_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).schoolholiday)
    state_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).stateholiday)
    holidays_this_week.append(school_holidays+state_holidays)
    index_list.append(index)
    
holidays_this_week = pd.Series(holidays_this_week)
holidays_this_week.shape

In [ ]:
holidays_last_week=[]
holidays_last_week_index=[]
for index, value in df_model.groupby(df_model['timestamp']).sum().iterrows():
    start_range = index - datetime.timedelta(days=7)
    end_range = index + datetime.timedelta(days=1)
    school_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).schoolholiday)
    state_holidays = sum((df_model.groupby(df['timestamp']).sum()[start_range:end_range]).stateholiday)
    holidays_last_week.append(school_holidays+state_holidays)
    holidays_last_week_index.append(index)
    
holidays_last_week = pd.Series(holidays_next_week)
holidays_last_week.shape

In [ ]:
temp_df = pd.DataFrame({'holidaysnextweek':holidays_next_week, 'timestamp': holidays_next_week_index})
df_model = pd.merge(df_model, temp_df, on=['timestamp'])

In [ ]:
temp_df = pd.DataFrame({'holidaysthisweek':holidays_this_week, 'timestamp': index_list})
df_model = pd.merge(df_model, temp_df, on=['timestamp'])

In [ ]:
temp_df = pd.DataFrame({'holidayslastweek':holidays_last_week, 'timestamp': holidays_last_week_index})
df_model = pd.merge(df_model, temp_df, on=['timestamp'])

In [ ]:
holidays_features = ['holidaysnextweek', 'holidaysthisweek', 'holidayslastweek']

features_x = list(set(features_x + holidays_features))

In [ ]:
print(df_model.shape)
df_model.head()

In [ ]:
print(df_model.columns)
print(features_x)

In [ ]:
features_x = ['open', 'store', 'storetype', 'holidayslastweek', '0_prominterval', 'stateholiday', 'assortment', 'dateint', 'holidaysthisweek', 'holidaysnextweek', 'promo', 'promosince', 'dayofweek', 'competitionopen', 'schoolholiday', 'competitiondistance']

# Predictive modelling

## XGBoost

For XGBoost and the RMSPE evaluation, no 0 values are permitted

In [ ]:
checkpoint = df_model.copy()

In [ ]:
df_model.sales = df_model.sales.apply(lambda x: np.nan if x == 0 else x)
df_model.loc[df_model['is_train'] == 1, 'saleslog'] = np.log(1+df_model.loc[df_model['is_train'] == 1]['sales'])

In [ ]:
#train test split
data = df_model.loc[(df_model['is_train'] == 1)]
x_train, x_test, y_train, y_test = train_test_split(data[features_x], 
                                                    data[features_y], 
                                                    test_size=0.2, 
                                                    random_state=3)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test, y_test)

num_round = 1000
evallist = [(dtrain, 'train'), (dtest, 'test')]

param = {'max_depth': 9,
         'eta': 0.01,
         'subsample': 0.75,
         'colsample_bytree': 0.6, 
         'objective': 'reg:squarederror',}

plst = list(param.items())

In [ ]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [ ]:
modto_csvxgb.train(plst, dtrain, num_round, evallist,
                  feval=rmspe_xg, verbose_eval=25, early_stopping_rounds=100)

In [ ]:
#Print Feature Importance
plt.figure(figsize=(18,8))
from xgboost import plot_importance
plot_importance(model)
plt.show()
plt.savefig('xgboost_feature_importance.png')

In [ ]:
# save dataframes on disk
df.to_csv('dataframe_raw', index=False)
df_model.to_csv('dataframe_raw_model', index=False)
data.to_csv('dataframe_modeldata', index=False)

In [ ]:
# save model using pickle
import pickle
filename = 'model_xgboost_01.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#make a submission dataframe to test RMSPE for unseen test-data (test.csv)
submit = df_model.loc[df_model['is_train'] == 0]
dsubmit = xgb.DMatrix(submit[features_x])
predictions = model.predict(dsubmit)

df_predictions = submit['id'].reset_index()
df_predictions['Id'] = df_predictions['id'].astype('int')
df_predictions['Sales'] = (np.exp(predictions) - 1) * 0.985 #Scale Back

df_predictions.sort_values('Id', inplace=True)
df_predictions[['Id', 'Sales']].to_csv('submit_xgboost_01.csv', index=False)